In [1]:
import pandas as pd
from Teacher import TeacherModel
from Student import StudentModel
from Interactions import Interaction,Experiment_multiturn,Interaction_single_turn, Experiment_single_turn

/workspace/venv_tfm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to determine the choice letter
def get_choice_letter(row):
    choices = ['q_op1', 'q_op2', 'q_op3','q_op4', 'q_op5']
    for choice in choices:
        if row[choice] == row['q_ans']:
            number = int(choice[-1])
            letter = chr(64 + number)
            return letter  # Return the last character which is the letter (A, B, C

In [3]:
def create_QA_set(df, i, reasoning = False):
    question = df["q_text"][i]
    options = "A: " + df["q_op1"][i] + "\n" + "B: " + df["q_op2"][i] + "\n"  + "C: " + df["q_op3"][i] + "\n"  + "D: " + df["q_op4"][i] + "\n"  + "E: " + df["q_op5"][i] + "\n" 
    answer = df["Answer_Letter"][i]
    
    if reasoning:
        reasoning_out = df["taskB"][i]
        return question,options,answer, reasoning_out

    else:
        return question,options,answer

In [4]:
def transform_dataframe(df,reasoning = False):
    # Create empty lists to store the transformed data
    questions, options, answers, reasoning_out = [], [], [], []

    # Iterate over each row in the DataFrame
    for i in range(len(df)):
        if reasoning:
            question, option, answer, reasoning_ = create_QA_set(df,i, reasoning)
            reasoning_out.append(reasoning_)
        else:
            question, option, answer = create_QA_set(df,i)
        
        questions.append(question)
        options.append(option)
        answers.append(answer)
    
    # Create a new DataFrame with the transformed data
    data = {
        'Question': questions,
        'Options': options,
        'Answer': answers
    }

    if reasoning:
        data['Reasoning'] = reasoning_out

    transformed_df = pd.DataFrame(data)
    return transformed_df

In [5]:
access_token = ""


In [6]:
df = pd.read_csv('data/cqa_data_test.csv')

In [7]:
# Apply the function to each row to create the new column
df['Answer_Letter'] = df.apply(get_choice_letter, axis=1)

In [8]:
df.head(2)

,Unnamed: 0,q_no,q_concept,q_text,q_op1,q_op2,q_op3,q_op4,q_op5,q_ans,taskA_pos,taskA_neg,taskB,Answer_Letter
0,0,26f7c29d5acb341f79610f7c040412a4,in charge of project,What kind of people will someone who is in cha...,greedy,worry,have knowledge,take charge,boss,have knowledge,"Have knowledge means information, understandin...",Greedy people have excessive desire and they a...,"Have knowledge means information, understandin...",C
1,1,c4f3db84c2782a9db26d543273e7ef50,disco,What town is known for disco?,rio de janeiro,new york,town,germany,hotel,rio de janeiro,Rio de Janeiro is a huge seaside city in brazi...,New York city is a city in New York state wher...,Rio de Janeiro is a huge seaside city in brazi...,A


In [9]:
question_1, options_1,answer_1, reasoning_1 = create_QA_set(df, 7, reasoning = True)
print(question_1)
print(options_1)
print(answer_1)
print(reasoning_1)
question_2, options_2,answer_2 = create_QA_set(df, 8)

Where does my body go after I am no longer living?
A: zombie
B: bodycam
C: coffin
D: graveyard
E: funeral

D
A person is said to be dead when they are not alive. Such bodies are called dead bodies and the specific place for the bodies to be buried is called a Graveyard. Coffin is not a place where dead bodies are buried while zombie, bodycam and funeral are not specific places.


In [10]:
df_qa = transform_dataframe(df)
df_qa_reas = transform_dataframe(df, reasoning = True)

In [11]:
df_qa.head(5)

,Question,Options,Answer
0,What kind of people will someone who is in cha...,A: greedy\nB: worry\nC: have knowledge\nD: tak...,C
1,What town is known for disco?,A: rio de janeiro\nB: new york\nC: town\nD: ge...,A
2,"The student was contemplating the problem, tha...",A: action\nB: discovery\nC: reflection\nD: dec...,B
3,What is mouthwash usually stored in?,A: this bottle will not fit in the medicine ca...,B
4,What helps someone be a good teacher?,A: teaching to the test\nB: vacation\nC: inter...,C


In [12]:
model_teacher_id = "microsoft/Phi-3-mini-128k-instruct"
#model_teacher_id = "/mistralai/Mistral-7B-v0.3"

model_student_id = "microsoft/Phi-3-mini-4k-instruct"

In [13]:
teacher = TeacherModel(access_token,model_teacher_id , attn_implementation = None)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
student = StudentModel(access_token,model_student_id )

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
interaction = Interaction(teacher, student,system_prompt_teacher = False,system_prompt_student = False)

In [16]:
interaction.make_first_interaction(question_1, options_1,answer_1,question_2, options_2,answer_2)

You are not running the flash-attention implementation, expect numerical differences.


Teacher: Question: Where does my body go after I am no longer living?
Choices: A: zombie, B: bodycam, C: coffin, D: graveyard, E: funeral
Answer: D: coffin
Reasoning: When a person passes away, their body is typically placed in a coffin, which is a box-like container designed to hold the deceased. This process is part of the funeral rites, which are ceremonies that honor and remember the life of the deceased. After the funeral, the coffin is usually taken to a cemetery, where it is buried in a grave. The coffin serves as a protective covering for the body, preserving it until the time of burial. Therefore, the correct answer is D: coffin, as it is the immediate resting place for the body after death.<|end|>
 Now, the question that you have to answer is: How is someone in a new relationship likely to feel about kissing?. 
 The choices are: A: like
B: hormones
C: shyness
D: happiness
E: pleasure



Student Answer: {
    "Reasoning": "When someone is in a new relationship, they are likely

False

In [17]:
#experiment = Experiment_single_turn(student,df_qa_reas[:200])

In [18]:
#experiment.complete_experiment_single_turn()

In [19]:
#experiment.df_results

In [20]:
experiment = Experiment_multiturn(teacher,student,df_qa[:200])

In [21]:
experiment.complete_experiment_multiturn()

Teacher: Question: What kind of people will someone who is in charge of a project look for?
Choices: A: greedy, B: worry, C: have knowledge, D: take charge, E: boss
Answer: C: have knowledge
Reasoning: When someone is in charge of a project, they need to find people who can contribute effectively to the project's success. This means they should look for individuals who have the necessary knowledge and skills to complete their tasks efficiently. While taking charge and being a boss are important qualities, they are not the primary focus when looking for team members. Greedy individuals may prioritize their own interests over the project's success, and worrying individuals may not be able to focus on the project's goals. Therefore, the best choice is C: have knowledge, as it ensures that the person in charge can build a competent and capable team.<|end|>
 Now, the question that you have to answer is: What town is known for disco?. 
 The choices are: A: rio de janeiro
B: new york
C: town


In [22]:
experiment.df_results

,Question,Correct_Ans,Answer_1,Answer_2
0,1.0,A,B,A
1,2.0,B,E,B
2,3.0,B,B,<NA>
3,4.0,C,C,<NA>
4,5.0,C,C,<NA>
...,...,...,...,...
194,195.0,A,A,<NA>
195,196.0,C,C,<NA>
196,197.0,B,B,<NA>
197,198.0,B,B,<NA>


In [24]:
experiment.df_results.to_csv("phi128_phi_multi.csv")